# Approximate Nearest Neighbor Search (ANN) with Hnswlib  (https://github.com/nmslib/hnswlib/).



In [1]:
!pip install sentence_transformers
!pip install pyvis
!pip install transformers_interpret
!pip install pyspark==3.5.0
!pip install torch
!pip install dask

# # !pip install pyngrok

!pip install memory_profiler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c7ab48b510e91d45cd16fcfd37671b624bf79f709b05127929b1920f7add50f1
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark:

In [8]:
pip install hnswlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=2287874 sha256=bbd628f571b1cd3cea06b77e15715044417c22aa034a9013cd8e26233f6f589e
  Stored in directory: /root/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib


In [17]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import pickle
import time
import hnswlib

In [18]:
import pandas as pd
df_claim_cpc_train = pd.read_csv("/content/drive/MyDrive/data_embedding/bigPatent_curated_550k.csv")
df_claim_cpc_test = pd.read_csv("/content/drive/MyDrive/data_embedding/df_claim_test_1M_pre_duplicates_removed_663.csv")



In [19]:
df_claim_cpc_train.rename(columns={'patent_id': 'id'}, inplace=True)


In [20]:
#import os
import numpy as np
import torch  # Import torch if not already imported

claim_embeddings = np.load("/content/drive/MyDrive/data_embedding/BigPatent_data_embeddings_550k.npy", allow_pickle=True)

In [21]:
%%time
#Loading the Patent SBERTa model from huggingface
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer
# Check if CUDA (GPU support) is available and set the device accordingly
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the model
model = SentenceTransformer("AI-Growth-Lab/PatentSBERTa").to(device)


Using device: cuda
CPU times: user 979 ms, sys: 328 ms, total: 1.31 s
Wall time: 2.97 s


In [22]:
%%time
query_text = "the present invention refers to a ventilation system used in hospital respiratory devices for the administration of anesthesia to newly born , pediatric and adult patients . more specifically , the present invention refers to a ventilation system that promotes a respiratory circuit with re - inhalation for the administration of anesthesia and that overcomes all inconveniences and deficiencies existing in respiratory devices of the state of the art . the ventilation system in respiratory devices comprises a bellows assembled within a reservoir , which is provided with a manifold with various gas exhaling valves , more specifically release valves to take out the excess of gases , and an exhaling valve for control gases . it is additionally constituted of a valve for the free flow of oxygen to quickly renew or replace the gases inside the respiratory circuit ."
query_embeddings = model.encode(query_text, convert_to_tensor=True, show_progress_bar=True)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 40.2 ms, sys: 3.43 ms, total: 43.7 ms
Wall time: 60.3 ms


In [23]:
query_embeddings.shape

torch.Size([768])

In [30]:
%%time

'''
from sentence_transformers import SentenceTransformer, util
claim_embeddings = util.normalize_embeddings(claim_embeddings)

claim_embeddings = claim_embeddings.to('cuda')
query_embeddings = util.normalize_embeddings(query_embeddings)

query_embeddings = query_embeddings.to('cuda')
hits = util.semantic_search(query_embeddings, claim_embeddings, score_function=util.dot_score)
'''

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs


"\nfrom sentence_transformers import SentenceTransformer, util\nclaim_embeddings = util.normalize_embeddings(claim_embeddings)\n\nclaim_embeddings = claim_embeddings.to('cuda')\nquery_embeddings = util.normalize_embeddings(query_embeddings)\n\nquery_embeddings = query_embeddings.to('cuda')\nhits = util.semantic_search(query_embeddings, claim_embeddings, score_function=util.dot_score)\n"

In [27]:
hits = util.semantic_search(query_embeddings, claim_embeddings, score_function=util.dot_score)

In [28]:
hits

[[{'corpus_id': 19974, 'score': 33.9092903137207},
  {'corpus_id': 106100, 'score': 33.059959411621094},
  {'corpus_id': 480013, 'score': 33.03443145751953},
  {'corpus_id': 123212, 'score': 32.91284942626953},
  {'corpus_id': 59694, 'score': 32.84141540527344},
  {'corpus_id': 64672, 'score': 32.71438217163086},
  {'corpus_id': 430289, 'score': 32.6430778503418},
  {'corpus_id': 377505, 'score': 32.5818977355957},
  {'corpus_id': 244966, 'score': 32.563438415527344},
  {'corpus_id': 261988, 'score': 32.471622467041016}]]

In [29]:
embedding_size = 768  # Adjust according to the embedding size of your model
top_k_hits = 10  # Output k hits


In [31]:
# Defining our hnswlib index
index_path = "./hnswlib.index"
index = hnswlib.Index(space="cosine", dim=embedding_size)


In [33]:
# Load or create the HNSWLIB index
if os.path.exists(index_path):
    print("Loading index...")
    index.load_index(index_path)
else:
    # Assuming 'corpus_embeddings' is a numpy array with shape (num_patents, embedding_size)
    corpus_embeddings = np.load("/content/drive/MyDrive/data_embedding/BigPatent_data_embeddings_550k.npy")  # Replace with the actual path to your embeddings

    print("Start creating HNSWLIB index")
    index.init_index(max_elements=len(corpus_embeddings), ef_construction=400, M=64)
    index.add_items(corpus_embeddings, list(range(len(corpus_embeddings))))
    print("Saving index to:", index_path)
    index.save_index(index_path)

Start creating HNSWLIB index
Saving index to: ./hnswlib.index


In [34]:
# Controlling the recall by setting ef
index.set_ef(50)  # ef should always be > top_k_hits


In [35]:
######### Search in the index ###########

print("Corpus loaded with {} patents".format(len(df_claim_cpc_train)))


Corpus loaded with 549164 patents


In [41]:

    #inp_query = ("Please enter a claim/query: ")

start_time = time.time()
    #query_embedding = model.encode(inp_query)
query_embeddings = query_embeddings.cpu()

    # We use hnswlib knn_query method to find the top_k_hits
corpus_ids, distances = index.knn_query(query_embeddings, k=top_k_hits)

# We extract corpus ids and scores for the first query
hits = [{"corpus_id": id, "score": 1 - score} for id, score in zip(corpus_ids[0], distances[0])]
hits = sorted(hits, key=lambda x: x["score"], reverse=True)
end_time = time.time()

#print("Input query:", inp_query)
print("Results (after {:.3f} seconds):".format(end_time - start_time))


Results (after 0.004 seconds):


In [42]:
hits

[{'corpus_id': 64672, 'score': 0.733490526676178},
 {'corpus_id': 19974, 'score': 0.7159314155578613},
 {'corpus_id': 480013, 'score': 0.7042564153671265},
 {'corpus_id': 123212, 'score': 0.7041542530059814},
 {'corpus_id': 106100, 'score': 0.70151686668396},
 {'corpus_id': 244966, 'score': 0.6938070058822632},
 {'corpus_id': 430289, 'score': 0.6932805776596069},
 {'corpus_id': 2324, 'score': 0.690913200378418},
 {'corpus_id': 59694, 'score': 0.6900128722190857},
 {'corpus_id': 209252, 'score': 0.6861035227775574}]

In [49]:
hits[0]

{'corpus_id': 64672, 'score': 0.733490526676178}

In [52]:
# Extract information about top similar patents
top_patent_indices = [int(hit['corpus_id']) for hit in hits]
top_similarity_scores = [hit['score'] for hit in hits]


In [53]:
top_patent_indices

[64672, 19974, 480013, 123212, 106100, 244966, 430289, 2324, 59694, 209252]

In [54]:
top_similarity_scores

[0.733490526676178,
 0.7159314155578613,
 0.7042564153671265,
 0.7041542530059814,
 0.70151686668396,
 0.6938070058822632,
 0.6932805776596069,
 0.690913200378418,
 0.6900128722190857,
 0.6861035227775574]

In [55]:
# Retrieve the details of the top similar patents from your database
top_similar_patents = df_claim_cpc_train.iloc[top_patent_indices]
top_similar_patents

,id,date,text
64672,9199049,2015-12-01,1. A ventilating system for providing inspirat...
19974,8925545,2015-01-06,1. A ventilation apparatus for treating sleep ...
480013,9849258,2017-12-26,1. A respiratory apparatus for delivery of a p...
123212,8881723,2014-11-11,1. A breathable gas supply apparatus for a pat...
106100,8677991,2014-03-25,1. A respirator or anesthesia apparatus for th...
244966,8420380,2013-04-16,1. A system for preserving a lung ex vivo comp...
430289,8973577,2015-03-10,1. A pressure support system comprising: a pre...
2324,9808606,2017-11-07,1. A valve system for an inflatable portion of...
59694,9308333,2016-04-12,1. Inhalation therapy device comprising: an ae...
209252,8875706,2014-11-04,1. A ventilator assembly for use in a ventilat...


In [56]:
top_similar_patents['similarity_score'] = top_similarity_scores
top_similar_patents

<ipython-input-56-1b4c3a69a10c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_similar_patents['similarity_score'] = top_similarity_scores


,id,date,text,similarity_score
64672,9199049,2015-12-01,1. A ventilating system for providing inspirat...,0.733491
19974,8925545,2015-01-06,1. A ventilation apparatus for treating sleep ...,0.715931
480013,9849258,2017-12-26,1. A respiratory apparatus for delivery of a p...,0.704256
123212,8881723,2014-11-11,1. A breathable gas supply apparatus for a pat...,0.704154
106100,8677991,2014-03-25,1. A respirator or anesthesia apparatus for th...,0.701517
244966,8420380,2013-04-16,1. A system for preserving a lung ex vivo comp...,0.693807
430289,8973577,2015-03-10,1. A pressure support system comprising: a pre...,0.693281
2324,9808606,2017-11-07,1. A valve system for an inflatable portion of...,0.690913
59694,9308333,2016-04-12,1. Inhalation therapy device comprising: an ae...,0.690013
209252,8875706,2014-11-04,1. A ventilator assembly for use in a ventilat...,0.686104
